# 概要

動的なニューラルネットワークモデルを作成する。<br>
これまで扱ったSequential APIや関数型APIは宣言的であり、静的な作成方法する方法だった。これら静的なモデル作成には多くのメリットがある。
例えばモデルの保存、クローン作成、共有、構造の表示、分析が簡単にできる。<br>
フレームワーク側で形の推測や型チェックを行え、訓練前にエラーを見つけることが出来る。
また、グラフ全体が静的なグラフなのでデバッグもしやすい。

一方、静的なモデル作成ではループ、形の変更、条件分岐などの動的な動作が必要な場合は対応できない。<br>
いま挙げたような命令型プログラミングのスタイルを扱うにはサブクラスAPIを使用する。

# サブクラスAPI

Modelクラスのサブクラスを定義する。<br>
コンストラクタの中に必要な層を定義し、callメソッドに各層での計算を実装する。

今回は関数型APIで実装した出力を複数持つワイド・アンド・ディープニューラルネットワークを実装する。

## パッケージインポート

In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

print(f'tensorflow ver.{tf.__version__}')
print(f'keras ver.{keras.__version__}')

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

tensorflow ver.2.8.0
keras ver.2.8.0


## データロードと前処理

###  データロード（sklearn.datasets.fetch_california_housing）

In [18]:
housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

### validation分割

In [19]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

# データサイズを確認
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_valid.shape : {x_valid.shape}')
print(f'y_valid.shape : {y_valid.shape}')

x_train.shape : (11610, 8)
y_train.shape : (11610,)
x_valid.shape : (3870, 8)
y_valid.shape : (3870,)


trainデータを確認

In [20]:
pd_x_train = pd.DataFrame(x_train, columns=housing.feature_names)
display(pd_x_train)
pd_x_train.info()
pd_x_train.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.7639,9.0,8.758621,1.724138,66.0,2.275862,39.04,-122.69
1,2.8812,23.0,5.064272,1.032136,1424.0,2.691871,39.93,-120.90
2,7.7360,21.0,7.000000,1.046921,913.0,2.677419,37.11,-121.95
3,4.9880,4.0,6.020027,1.049399,4438.0,2.962617,32.95,-117.12
4,2.7159,45.0,5.306590,1.126074,1132.0,3.243553,37.77,-122.19
...,...,...,...,...,...,...,...,...
11605,4.9028,14.0,6.372152,1.008861,2124.0,2.688608,37.10,-121.63
11606,4.5300,22.0,5.181195,1.032321,2812.0,2.754163,33.83,-118.06
11607,3.6125,15.0,5.917808,1.191781,208.0,2.849315,38.53,-120.59
11608,2.4659,45.0,5.573668,1.062696,839.0,2.630094,38.35,-121.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11610 entries, 0 to 11609
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      11610 non-null  float64
 1   HouseAge    11610 non-null  float64
 2   AveRooms    11610 non-null  float64
 3   AveBedrms   11610 non-null  float64
 4   Population  11610 non-null  float64
 5   AveOccup    11610 non-null  float64
 6   Latitude    11610 non-null  float64
 7   Longitude   11610 non-null  float64
dtypes: float64(8)
memory usage: 725.8 KB


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000
mean,3.871197,28.535573,5.418077,1.094453,1426.556503,3.047560,35.626552,-119.567388
std,1.895562,12.583297,2.408378,0.483486,1104.296288,11.555917,2.123165,1.997852
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.560000,-124.350000
25%,2.573050,18.000000,4.440930,1.005722,788.000000,2.422020,33.930000,-121.780000
50%,3.539100,29.000000,5.219590,1.048309,1175.000000,2.816946,34.260000,-118.500000
75%,4.729300,37.000000,6.036542,1.098186,1726.000000,3.280752,37.710000,-118.010000
max,15.000100,52.000000,132.533333,34.066667,28566.000000,1243.333333,41.920000,-114.310000


### 前処理

#### スケーリング

### [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
データの標準化を行う。

代表的なメソッドは以下：

|メソッド|説明|
|---|---|
|fit()|標準化するための平均と分散を計算する。|
|trasform()|（事前に計算した平均と分散を使用して）標準化を行う。|
|fit_transform()|平均と分散を計算し、標準化を行う。|

In [21]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)  # x_trainの平均・分散を使用する（のはなぜ？）
x_test = scaler.transform(x_test)  # x_trainの平均・分散を使用する（のはなぜ？）

#### inputデータ分割

In [22]:
# ワイドパス、ディープパス用で分割
x_train_A, x_train_B = x_train[:, :5], x_train[:, 2:]
x_valid_A, x_valid_B = x_valid[:, :5], x_valid[:, 2:]
x_test_A, x_test_B = x_test[:, :5], x_test[:, 2:]

## モデル作成

### レイヤ構成を定義

In [23]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        # super().__init__(**kwargs)  # handles standard args (ex : name)
        super().__init__(**kwargs)  # 標準引数(ex : name)を処理する←これ2回いる？
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)  # keras.Model.output属性があるので競合しないように
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden_1 = self.hidden1(input_B)
        hidden_2 = self.hidden2(hidden_1)
        concat = keras.layers.concatenate([input_A, hidden_2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden_2)
        return main_output, aux_output

model = WideAndDeepModel(units=30, activation='relu')


この例では関数型APIとよく似ているが、input層を作る必要がないところが異なる。<br>
またコンストラクタ内で層を作成しcallメソッドの中で層の利用の仕方を実装するため、層の作成と利用を分離出来る。<br>
これにより関数型APIよりも柔軟にモデル作成が出来る。

## モデルのコンパイル

柔軟さの代償もある。<br>
モデルのアーキテクチャがcallメソッドに隠されてしまうため、Kerasが簡単にモデルを精査できなくなる。<br>
また、保存やクローン作成もできなくなる。<br>
summaryメソッドを呼び出しても層のリストは得られるが接続情報は得られない。

In [33]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD())

model.summary()  # errorになる

Model: "wide_and_deep_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             multiple                  210       
                                                                 
 dense_5 (Dense)             multiple                  930       
                                                                 
 dense_6 (Dense)             multiple                  36        
                                                                 
 dense_7 (Dense)             multiple                  31        
                                                                 
Total params: 1,207
Trainable params: 1,207
Non-trainable params: 0
_________________________________________________________________


# 学習と評価

### コールバック設定

In [25]:
# 早期打ち切りのコールバック関数
# 学習打ち切り時に性能が最高だった時の重みを復元するので最良モデルの保存と復元は不要
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)  # patienceで指定したエポック数学習が進まなかったときに学習を打ち切る

In [26]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

print_valid_train_ration_cb = PrintValTrainRatioCallback()

In [27]:
import os

# ログ出力のルートディレクトリ
root_dir = os.path.join(os.curdir, 'my_logs')

# ログディレクトリ名を生成する関数
def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

## 学習

学習時もそれぞれの出力に対してラベルを指定する必要がある。<br>
今回の例では両方同じものを予測しているので同じラベルを渡す。

In [28]:
history = model.fit([x_train_A, x_train_B], [y_train, y_train], epochs=10, validation_data=([x_valid_A, x_valid_B], [y_valid, y_valid]), callbacks=[tensorboard_cb, early_stopping_cb, print_valid_train_ration_cb])

Epoch 1/10
359/363 [============================>.] - ETA: 0s - loss: 1.5777 - output_1_loss: 1.5249 - output_2_loss: 2.0524
val/train: 1.09
363/363 [==============================] - 1s 3ms/step - loss: 1.5793 - output_1_loss: 1.5275 - output_2_loss: 2.0458 - val_loss: 1.7264 - val_output_1_loss: 1.7692 - val_output_2_loss: 1.3417
Epoch 2/10
357/363 [============================>.] - ETA: 0s - loss: 0.8197 - output_1_loss: 0.7922 - output_2_loss: 1.0670
val/train: 0.72
363/363 [==============================] - 1s 2ms/step - loss: 0.8151 - output_1_loss: 0.7873 - output_2_loss: 1.0656 - val_loss: 0.5877 - val_output_1_loss: 0.5385 - val_output_2_loss: 1.0311
Epoch 3/10
343/363 [===========================>..] - ETA: 0s - loss: 0.5381 - output_1_loss: 0.4968 - output_2_loss: 0.9101
val/train: 1.08
363/363 [==============================] - 0s 1ms/step - loss: 0.5378 - output_1_loss: 0.4963 - output_2_loss: 0.9113 - val_loss: 0.5823 - val_output_1_loss: 0.5486 - val_output_2_loss: 0.885

## 学習結果の可視化

In [29]:
%load_ext tensorboard
%tensorboard --logdir ./my_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 59349), started 0:33:31 ago. (Use '!kill 59349' to kill it.)

In [30]:
# import pandas as pd
# import matplotlib.pyplot as plt
# pd.DataFrame(history.history).plot(figsize=(8, 5))  # historyはエポック毎のloss, val_lossを保持する
# plt.grid(True)
# plt.gca().set_ylim(0, 1) # 縦の範囲を 0 から 1 までに
# plt.show()

## 学習結果の評価

In [31]:
total_loss, main_loss, aux_loss = model.evaluate((x_test_A, x_test_B), (y_test, y_test))
total_loss, main_loss, aux_loss

162/162 [==============================] - 0s 914us/step - loss: 0.4322 - output_1_loss: 0.4120 - output_2_loss: 0.6140


(0.4322028160095215, 0.4120028614997864, 0.6140033602714539)

# 学習済みモデルを使った予測

In [32]:
# サンプル用にデータサイズを限定
x_new_A, x_new_B = x_test_A[:3], x_test_B[:3]

y_pred_main, y_pred_aux = model.predict((x_new_A, x_new_B))
print(f'predict_main : {y_pred_main.reshape(-1)}')
print(f'predict_aux : {y_pred_aux.reshape(-1)}')
print(f'correct : {y_test[:3]}')

predict_main : [1.7976997 1.4637512 2.6465533]
predict_aux : [1.8732376 1.2310947 2.2673492]
correct : [1.663 1.144 3.5  ]
